Importamos los datos desde kaggle, creando la carpeta de data, con los datos comprmidos ( como se descarga por defecto de kaggle), y ha continuacion descomprimimos las carpetas con las imagenes

Versión de Colab

In [ ]:
# !pip install kaggle

# from google.colab import files
# import os, zipfile
# from glob import glob

# # Subir kaggle.json
# files.upload()
# os.makedirs("/root/.kaggle", exist_ok=True)
# !mv kaggle.json /root/.kaggle/
# !chmod 600 /root/.kaggle/kaggle.json

# # Descargar
# os.makedirs("data/raw", exist_ok=True)
# !kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification -p data/raw


Versión en local

In [7]:
import os
import zipfile


if not os.path.exists("data/raw"):
    os.makedirs("data/raw")

#Descarga de datos desde kaggle
!kaggle datasets download -d gunavenkatdoddi/eye-diseases-classification -p data/raw

#Descomprimir zip
with zipfile.ZipFile("data/raw/eye-diseases-classification.zip", 'r') as zip_ref:
    zip_ref.extractall("data/raw")

Dataset URL: https://www.kaggle.com/datasets/gunavenkatdoddi/eye-diseases-classification
License(s): ODbL-1.0
eye-diseases-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


## Preprocesamiento de los datos

In [ ]:
import os
from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm

path = "data/raw/dataset"  

imgs = []
labels = []
folders = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]

for folder in tqdm(folders):
    folder_path = os.path.join(path, folder) #Creamos la direccion de cada carpeta
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname) #obtenemos direccion de cada archivo
        try:
            with Image.open(fpath) as img: 
                img = img.convert("RGB") # Lectura de image
                arr = np.array(img)
            imgs.append(arr)
            labels.append(folder)
        except Exception as e:
            print("skip:", fpath, str(e))

x = pd.DataFrame({"image": imgs})
y = pd.DataFrame({"labels":labels})

100%|██████████| 4/4 [00:35<00:00,  8.94s/it]

n imágenes: 4217 n etiquetas: 4217
ejemplo shape imagen: (512, 512, 3)
